# Week 7 Machine Learning
- Scott Williamson
- 6-13-24
- https://github.com/scottryanw/datafun-07-ml

##  Introduction
Work with Machine Learning principles to build models, make predictions, and visualize data using pandas, seaborn, matplotlib, and scikit-learn. Models will use simple linear regression to predict future values. 

Completed as part of Data Analytics Fundaments course towards completion of Data Analytics Masters at NW Missouri State University.

### Import Dependencies

In [1]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import sklearn
from sklearn import metrics
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.model_selection import train_test_split

## Part 1 - Chart a Straight Line